In [1]:
import pandas as pd
import numpy as np

Считываем данные и дропаем ненужные колонки

In [2]:
data = pd.read_csv('../data/data.csv')
data.drop(['campaign_clicks', 'rate0', 'rate1', 'impressions', 'oaid_hash'], axis=1, inplace=True)
data.head()

,date_time,zone_id,banner_id,os_id,country_id,banner_id0,g0,coeff_sum0,banner_id1,g1,coeff_sum1,clicks
0,2021-09-27 00:01:30.000000,0,0,0,0,1240,0.035016,-7.268846,0,0.049516,-5.369901,1
1,2021-09-26 22:54:49.000000,1,1,0,1,1,0.054298,-2.657477,269,0.031942,-4.449220,1
2,2021-09-26 23:57:20.000000,2,2,0,0,2,0.014096,-3.824875,21,0.014906,-3.939309,1
3,2021-09-27 00:04:30.000000,3,3,1,1,3,0.015232,-3.461357,99,0.050671,-3.418403,1
4,2021-09-27 00:06:21.000000,4,4,1,0,4,0.051265,-4.009026,11464230,0.032005,-2.828797,1


Преобразуем дату

In [3]:
def analysis(data: pd.DataFrame):
    data['date_time'] = pd.to_datetime(data['date_time'])

analysis(data)

Разделяем на трейн и тест по дате. От трейна берём один процент, чтобы быстрее обучалось. Делаем три копии теста: одна для подсчёта pi остаётся дата фреймом, две другие для модели с banner_id = banner_id0 и banner_id1 соответственно.

In [4]:
from sklearn.preprocessing import OneHotEncoder

def feature_engineering(data: pd.DataFrame) -> (pd.DataFrame, pd.DataFrame, pd.DataFrame, pd.DataFrame):
    train = data[data['date_time'] < '2021-10-02'].sample(frac=0.01, random_state=1337)
    train_clicks = train['clicks']
    test = data[data['date_time'] >= '2021-10-02']
    test = test[test['banner_id0'] == test['banner_id']]
    test_for_ips = test.copy()
    test_clicks = test['clicks']
    train = train.drop(['date_time', 'clicks'], axis=1)
    test = test.drop(['date_time', 'clicks'], axis=1)
    test_1 = test.copy()
    test_1['banner_id'] = test_1['banner_id1']
    ohe = OneHotEncoder(handle_unknown='ignore')
    train = ohe.fit_transform(train)
    test = ohe.transform(test)
    test_1 = ohe.transform(test_1)
    return train, train_clicks, test, test_1, test_clicks, test_for_ips

train_x, train_y, test_x_0, test_x_1, test_y, test_for_ips = feature_engineering(data)

Проверяем, что все данные имеют ожидаемый размер.

In [5]:
print(data.head())
print(data.shape)
print(train_y.shape)
print(train_x.shape)
print(test_for_ips.shape)
print(test_y.shape)

            date_time  zone_id  banner_id  os_id  country_id  banner_id0  \
0 2021-09-27 00:01:30        0          0      0           0        1240   
1 2021-09-26 22:54:49        1          1      0           1           1   
2 2021-09-26 23:57:20        2          2      0           0           2   
3 2021-09-27 00:04:30        3          3      1           1           3   
4 2021-09-27 00:06:21        4          4      1           0           4   

         g0  coeff_sum0  banner_id1        g1  coeff_sum1  clicks  
0  0.035016   -7.268846           0  0.049516   -5.369901       1  
1  0.054298   -2.657477         269  0.031942   -4.449220       1  
2  0.014096   -3.824875          21  0.014906   -3.939309       1  
3  0.015232   -3.461357          99  0.050671   -3.418403       1  
4  0.051265   -4.009026    11464230  0.032005   -2.828797       1  
(15821472, 12)
(136925,)
(136925, 542705)
(1890562, 12)
(1890562,)


Используем LogisticRegression, который упоминался в лекции, вместе с liblinear - не SGD-like оптимизатором.

In [6]:
from sklearn.linear_model import LogisticRegression

def create_model(penalty, C):
    return LogisticRegression(C = C, penalty=penalty, solver='liblinear')

Проверяем, что с моделью всё нормально

In [7]:
from sklearn.metrics import roc_auc_score, log_loss

model = create_model(penalty='l2', C=1).fit(train_x, train_y)
y_pred_proba = model.predict_proba(test_x_0)[:, 1]
roc_auc_metric = roc_auc_score(test_y, y_pred_proba)
log_loss_metric = log_loss(test_y, y_pred_proba)
print(f"roc_auc={roc_auc_metric}, log_loss={log_loss_metric}")

roc_auc=0.7702231730484378, log_loss=0.1394408753167321


P(N_1 > N_2) = P(N_1 - N_2 > 0) = 1 - F_{N_1 - N_2}(0). N_1 - N_2 - тоже нормальное со средним как разность средних и квадратом стандартного отклонения как сумма квадратов стандартных отклонений. Также добавим немного, чтобы стандартное отклонение было не 0

In [8]:
from scipy.stats import norm

def calc_pi(coeff0, g0, coeff1, g1):
    return 1 - norm.cdf(0, loc=coeff0 - coeff1, scale=np.sqrt(g0 ** 2 + g1 ** 2) + 1e-6)

pi_0 = calc_pi(test_for_ips['coeff_sum0'], test_for_ips['g0'], test_for_ips['coeff_sum1'], test_for_ips['g1'])

Считаем новые коэффы с помощью logit и также подсчитываем pi_1

In [9]:
from scipy.special import logit

test_for_ips['coeff_sum0_new'] = logit(model.predict_proba(test_x_0)[:, 1])
test_for_ips['coeff_sum1_new'] = logit(model.predict_proba(test_x_1)[:, 1])

pi_1 = calc_pi(test_for_ips['coeff_sum0_new'], test_for_ips['g0'], test_for_ips['coeff_sum1_new'], test_for_ips['g1'])

Чтобы не делить на 0, добавим немного к pi_0

In [10]:
cips = np.mean(test_y * np.minimum(pi_1 / (pi_0 + 1e-10), 10))
print(cips)

0.07195291109093356
